#### This notebook is now primarily used as a scrap notebook for quick code-testing purposes.

In [ ]:
# TODO: Replace prints with logging.

In [17]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

from fake_useragent import UserAgent

import os
import time
from subprocess import Popen

import re
import string

import pandas as pd

In [18]:
import pandas as pd
import numpy as np

import re
import string

In [19]:
import requests
from torpy.http.requests import tor_requests_session
from selenium import webdriver
from bs4 import BeautifulSoup


In [20]:
import time
from fake_useragent import UserAgent

In [21]:
from subprocess import Popen

In [22]:
import os

def restart_tor():
    '''
    A forceful hack since tor doesn't function as intended on my windows and none of the solution work. Too bad! https://www.youtube.com/watch?v=k238XpMMn38
    
    Commented out code is how it should work.
    
    TODO: Descr.
    '''
    tor_path = 'C:/Users/Jurgis/Desktop/tor-win32-0.4.2.7/Tor/tor.exe'
    
    # Kill existing tor process. TODO: Make this OS-proof.
    os.system("taskkill /F /im tor.exe")
    
    # Run a tor process in the background, wait for it to start.
    Popen(tor_path, shell=True)
    
    time.sleep(2)
    pass



# from stem import Signal
# from stem.control import Controller

# # signal TOR for a new connection 
# def renew_connection():
#     with Controller.from_port(port = 9051) as controller:
#         controller.authenticate(password="eatshit")
#         controller.signal(Signal.NEWNYM)


In [23]:
def get_tor_session():
    # Use Tor ports for a requests.session.
    session = requests.session()
    session.proxies = {
        'http':  'socks5://127.0.0.1:9050',
        'https': 'socks5://127.0.0.1:9050'
    }
    
    # Set additional Tor session parameters.
    s.headers = UserAgent().random
    
    print('Built a Tor session at IP {}.'.format(session.get('http://httpbin.org/ip').json()['origin']))
    return session

In [24]:
def get_number_of_pages(session, url):
    '''
    TODO: Descr.
    TODO: Handle SPECIFIC banned / captcha'ed cases with try excepts.
    '''
    # Set additional Tor session parameters.
    session.headers = UserAgent().random

    # Scrape the main page to get the total number of pages.
    page = session.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    # Extract all page number buttons to find the maximum digit value.
    page_number_buttons = str(soup.find_all(class_='page-bt'))
    page_number_buttons = re.findall('\d+', page_number_buttons)

    total_pages = max([int(number) for number in page_number_buttons])

    return total_pages

In [25]:
def get_page_urls(session, url):
    '''
    TODO: Descr.
    TODO: Handle SPECIFIC exceptions by restarting a tor session and re-running.
    '''
    try:
        # Get page contents and put them in a "soup".
        page = session.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')

        # Extract all listings and get their urls.
        listings = soup.find_all(class_='list-adress')
        listings_urls = [listing.find('a', href=True)['href'] for listing in listings]
        
        return listings_urls
    
    except Exception as e:
        return e

In [26]:
s = get_tor_session()
total_pages = get_number_of_pages('https://www.aruodas.lt/butu-nuoma/', s)

# Get listing urls for all of the pages. If an exception occurs while scraping, prints the exception.
listing_urls = []
for page_number in range(1, total_pages + 1):
    # TODO: Move to config.
    page_url = f'https://www.aruodas.lt/butu-nuoma/puslapis/{page_number}/?FOrder=AddDate&detailed_search=1'
    
    page_urls = get_page_urls(s, page_url)
    if type(page_results) == list:
        listing_urls.extend(page_urls)
    else:
        print(page_results)
        
# Save urls to a file? TODO: Is this necessary?
with open('../data/listing_urls.txt', 'w') as f:
    f.write('\n'.join(listing_urls))

NameError: name 's' is not defined

In [ ]:
def get_object_data(soup, html_tags=None):
    '''
    soup - BS4 object.
    
    TODO: Create env variable for html tags or save them to a file somehow.
    '''
    
    if html_tags is None:
        pass
    
    # Initialize HTML class, id, etc. tags.
    html_tag_listing_statistics = 'obj-top-stats'

    html_tag_building_energy_class = 'energy-class-tooltip'

    html_tag_object_details = ['obj-details', 'obj-details '] # Weird, but necessary.
    html_tag_object_details_names = 'dt'
    html_tag_object_details_items = 'dd'
    html_tag_object_description = 'collapsedText'
    html_tag_object_name = 'obj-header-text'

    html_tag_neighbourhood_statistics = 'advertStatisticHolder'
    html_tag_neighbourhood_statistics_names = 'cell-text'
    html_tag_neighbourhood_statistics_items = 'cell-data'

    html_tag_realtor = 'obj-contacts'
    html_tag_realtor_name = 'contacts-title'
    html_tag_no_realtor = 'obj-contacts simple'
    
    
    # Find all name and item classes withing object details class.
    object_details_class = soup.find(class_=html_tag_object_details[0])
    
    # Sometimes listings get displayed but have no info. Scraper catchers? TODO: Investigate.
    if object_details_class is None:
        return None
        
    
    object_names = object_details_class.find_all(html_tag_object_details_names)
    object_items = object_details_class.find_all(html_tag_object_details_items)

    # Get field names.
    object_names = [item.contents[0] for item in object_names]

    # Extract values from item fields.
    object_item_values = []
    for object_item in object_items:

        # Single value items.
        if len(object_item) == 1:
            object_item_values.append(object_item.contents[0])

        # Multiple value items.
        elif len(object_item) > 1:

            # Tries to get list-like items.
            items = [item.contents[0] for item in object_item.find_all(class_='special-comma')]
            if len(items) != 0:
                object_item_values.append(items)

            # If the list is empty, the item is most likely single value (hence index 0), with additional ads, links, etc.
            else:
                object_item_values.append(object_item.contents[0])

    # Set everything as a dictionary.
    object_data = dict(zip(object_names, object_item_values))


    # Find all name and item classes withing object details class.
    neighbourhood_statistics = soup.find(id=html_tag_neighbourhood_statistics)

    neighbourhood_statistics_names = neighbourhood_statistics.find_all(class_=html_tag_neighbourhood_statistics_names)
    neighbourhood_statistics_items = neighbourhood_statistics.find_all(class_=html_tag_neighbourhood_statistics_items)

    # Extract the name and the item values, add them to the dictionary.
    neighbourhood_statistics_names = [item.contents[0] for item in neighbourhood_statistics_names]
    neighbourhood_statistics_items = [item.contents[0] for item in neighbourhood_statistics_items]

    for name, item in zip(neighbourhood_statistics_names, neighbourhood_statistics_items):
        object_data[name] = item

    # Get object listing statistics (views, favorites, etc.).
    listing_statistics = soup.find(class_=html_tag_listing_statistics).contents

    object_data[listing_statistics[0]] = listing_statistics[1].contents[0]
    
    try:
        object_data['Listing Favorites'] = listing_statistics[3].contents[0]
    except:
        object_data['Listing Favorites'] = 0

    # Get object description text.
    object_data['Object Description'] = soup.find(id=html_tag_object_description).contents

    # Get object name - City, Neighbourhood, Street, Other.
    object_data['Listing Name'] = soup.find(class_=html_tag_object_name).contents[0]

    # Get energy class rating.
    building_energy_class = soup.find(class_=html_tag_building_energy_class)

    try:
        object_data['Building Energy Class'] = building_energy_class.contents[1]
        object_data['Building Energy Class Category'] = building_energy_class.contents[2]
    except:
        object_data['Building Energy Class'] = np.nan
        object_data['Building Energy Class Category'] = np.nan

    # Check if a realtor is present, get it's name if it is.
    object_contacts = soup.find(class_=html_tag_no_realtor)
    if object_contacts is None:
        realtor_name = soup.find(class_=html_tag_realtor).find(class_=html_tag_realtor_name).contents[0]
        # Handles cases where the realtor is a company, not a person. TODO: Add RealtorCompany variable.
        if realtor_name == 'Pardavėjo kontaktai':
            object_data['Realtor Name'] = np.nan
            object_data['Realtor'] = False
        else:
            object_data['Realtor Name'] = realtor_name
            object_data['Realtor'] = True
    else:
        object_data['Realtor Name'] = np.nan
        object_data['Realtor'] = False

    return object_data

In [ ]:
def parse_object_data(object_data):
    '''
    TODO: Descr.
    '''
    variables_integer = ['Plotas', 'KainaMėn', 'KambariųSk', 'Aukštas', 'AukštųSk', 'Metai', 'ArtimiausiasDarželis', 'ArtimiausiaMokymoĮstaiga', 'ArtimiausiaParduotuvė', 'ViešojoTransportoStotelė', 'Nusikaltimai500MSpinduliuPraėjusįMėnesį']
    variables_categorical = ['PastatoTipas', 'Šildymas', 'Įrengimas']
    variables_lists = ['Ypatybės', 'PapildomosPatalpos', 'PapildomaĮranga', 'Apsauga']
    
    
    # Transform keys from whatever messy format to VariableName.
    keys = list(object_data.keys())
    keys = [key.strip() for key in keys]
    keys = [key.translate(str.maketrans('', '', string.punctuation)) for key in keys]
    keys = [''.join(key.lower().title().split(' ')) for key in keys]

    object_data = dict(zip(keys, list(object_data.values())))
    
    
    # Convert integer variables to integer.
    for variable in variables_integer:
        # Handles cases when variable is missing:
        try:
            object_data[variable] = int(re.findall('\d+', object_data[variable])[0])
        except:
            continue

    # Remove whitespace from categorical variables.
    for variable in variables_categorical:
        object_data[variable] = object_data[variable].strip()


    # Transform lists of variables into pseudo categorical variables. E.g. Variable: ['feat1', 'feat2'] -> Variable_feat1: 1, Variable_feat2: 1.
    for variable in variables_lists:
        # Convert space delimited text to TitleCamelCase. E.g. 'This house' -> 'ThisHouse'.
        object_data[variable] = [''.join(item.title().split(' ')) for item in object_data[variable]]

        # Create a pseudo categorical variable where multiple values in a category is present.
        for feature in object_data[variable]:
            object_data[variable + '_' + feature] = 1 


    # Split ListingName into City, Neighbourhood, Street.
    object_data['ListingName'] = object_data['ListingName'].split(',')
    object_data['BuildingCity'] = object_data['ListingName'][0].strip()
    object_data['BuildingNeighbourhood'] = object_data['ListingName'][1].strip()
    object_data['BuildingStreet'] = object_data['ListingName'][2].strip()


    # Split Total/Today views into separate variables.
    object_data['SkelbimąPeržiūrėjoIšVisošiandien'] = object_data['SkelbimąPeržiūrėjoIšVisošiandien'].split('/')
    object_data['ListingViewsTotal'] = int(object_data['SkelbimąPeržiūrėjoIšVisošiandien'][0])
    object_data['ListingViewsToday'] = int(object_data['SkelbimąPeržiūrėjoIšVisošiandien'][1])


    # Transform ObjectDescription into a single string, as well as replacing <br/>'s with \n.
    object_data['ObjectDescription'] = ''.join([str(item) for item in object_data['ObjectDescription']])
    object_data['ObjectDescription'] = re.sub('<br/s*?>', '\n', object_data['ObjectDescription'])


    # Drop no longer required variables.
    variables_drop = []
    variables_drop.extend(variables_lists)
    variables_drop.extend(['ListingName', 'SkelbimąPeržiūrėjoIšVisošiandien'])
    for variable in variables_drop:
        object_data.pop(variable)
    
    
    return object_data

In [33]:
driver = webdriver.Chrome('D:/Projects/Personal/Current/Rent AVM/rent-avm/scraper/chromedriver.exe')

driver.get('https://www.aruodas.lt/butu-nuoma-kaune-centre-k-bugos-g-centras-k-bugos-g-vieno-kambario-46-kvm-4-989277/')
page_source = driver.page_source

soup = BeautifulSoup(page_source, 'lxml')

object_details_class = soup.find(class_=["obj-details", "obj-details "])

In [32]:

page_source = driver.page_source

In [290]:
with open('../data/listing_urls.txt', 'r') as f:
    listing_urls = f.readlines()
    
res = []
for listing_url in listing_urls[1:5]:
    driver = webdriver.Chrome('D:/Projects/Personal/Current/Rent AVM/rent-avm/scraper/chromedriver.exe')

    driver.get(listing_url)
    page_source = driver.page_source
    driver.close()

    soup = BeautifulSoup(page_source, 'lxml')

    res.append(get_object_data(soup))
    

df = pd.DataFrame()
for item in res:
    df = df.append(pd.DataFrame(parse_object_data(item), index=[0]))
    
pd.set_option('display.max_columns', 500)
df

,Plotas,KainaMėn,KambariųSk,Aukštas,AukštųSk,Metai,PastatoTipas,Šildymas,Įrengimas,ArtimiausiasDarželis,ArtimiausiaMokymoĮstaiga,ArtimiausiaParduotuvė,ViešojoTransportoStotelė,Nusikaltimai500MSpinduliuPraėjusįMėnesį,ListingFavorites,ObjectDescription,BuildingEnergyClass,BuildingEnergyClassCategory,RealtorName,Realtor,Ypatybės_UždarasKiemas,Ypatybės_Internetas,Ypatybės_AukštosLubos,Ypatybės_NaujaKanalizacija,Ypatybės_NaujaElektrosInstaliacija,Ypatybės_VirtuvėSujungtaSuKambariu,PapildomosPatalpos_Balkonas,PapildomosPatalpos_VietaAutomobiliui,PapildomaĮranga_SuBaldais,PapildomaĮranga_Kondicionierius,PapildomaĮranga_SkalbimoMašina,PapildomaĮranga_Šaldytuvas,PapildomaĮranga_VirtuvėsKomplektas,PapildomaĮranga_Viryklė,PapildomaĮranga_Indaplovė,PapildomaĮranga_Vonia,Apsauga_ŠarvuotosDurys,Apsauga_Signalizacija,Apsauga_KodinėLaiptinėsSpyna,BuildingCity,BuildingNeighbourhood,BuildingStreet,ListingViewsTotal,ListingViewsToday,PapildomosPatalpos_Drabužinė,PapildomaĮranga_ŠildomosGrindys,PapildomaĮranga_DušoKabina,Apsauga_Videokameros,Apsauga_BudintisSargas,NamoNumeris,ButoNumeris,VidutiniškaiTiekKainuotųŠildymas1Mėn,Ypatybės_TualetasIrVoniaAtskirai,Ypatybės_KabelinėTelevizija,PapildomosPatalpos_Sandėliukas,PapildomosPatalpos_Terasa,PapildomosPatalpos_Rūsys,PapildomaĮranga_PlastikiniaiVamzdžiai,Ypatybės_ButasPalėpėje,PapildomosPatalpos_YraPalėpė
0,110,990,3,2,2,2005,Mūrinis,"Centrinis kolektorinis, centrinis",Įrengtas,210.0,190.0,270.0,200.0,0.0,0,\n Išskirtinis butas prestižinė...,NaN,NaN,NaN,False,1,1,1.0,1.0,1.0,1,1.0,1.0,1,1.0,1,1,1,1,1,1,1,1.0,1.0,Vilnius,Senamiestis,Dysnos g.,36,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,103,899,3,11,27,2004,Monolitinis,Centrinis kolektorinis,Įrengtas,NaN,NaN,NaN,NaN,NaN,0,\n -\tJOKIŲ TARPININKAVIMO MOKE...,NaN,NaN,Akorus Real Estate,True,1,1,NaN,1.0,1.0,1,1.0,1.0,1,1.0,1,1,1,1,1,1,1,1.0,1.0,Vilnius,Šnipiškės,Konstitucijos pr.,48,48,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,113,1,4,7,9,2010,Mūrinis,Centrinis kolektorinis,Įrengtas,290.0,130.0,130.0,NaN,0.0,6,\n Išnuomosiu 4 kambarių 113 kv...,NaN,NaN,NaN,False,1,1,NaN,1.0,1.0,1,1.0,1.0,1,1.0,1,1,1,1,1,1,1,1.0,1.0,Kaunas,Centras,K. Donelaičio g.,5195,122,1.0,NaN,1.0,1.0,NaN,\n 81A ...,\n 51 ...,"\n 19,21 €/mėn.",1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN
0,54,500,1,3,3,1940,Mūrinis,Dujinis,Įrengtas,100.0,140.0,190.0,170.0,0.0,2,"\n Erdvus, saulėtas, nestadarti...",NaN,NaN,NaN,False,1,1,1.0,NaN,NaN,1,NaN,NaN,1,NaN,1,1,1,1,1,1,1,NaN,NaN,Vilnius,Senamiestis,Šv. Stepono g.,732,732,NaN,1.0,NaN,NaN,NaN,\n 9 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0


In [13]:

import pandas as pd




test = [
    {'col1': 1, 'col2': 1                       },
    {'col1': 2,             'col3': 2, 'col4': 2},
    {           'col2': 3                       }
]


df = pd.DataFrame()

pd.DataFrame(map(pd.Series, test))


,col1,col2,col3,col4
0,1.0,1.0,NaN,NaN
1,2.0,NaN,2.0,2.0
2,NaN,3.0,NaN,NaN


In [16]:
correct_output = False
retries = 0
while (correct_output) or (retries < 3):
    retries = retries + 1
    print(retries)

1
2
3


In [80]:
'contacts-title' - realtor name exists
'obj-contacts ', 'obj-contacts' - realtor exists / no realtor
'obj-contacts simple' - individual
'long-link-wrapper' - link exists

In [18]:
import os


'D:\\Projects\\Personal\\Current\\Rent AVM\\rent-avm\\scraper'

In [ ]:
#driver = webdriver.Chrome('D:/Projects/Personal/Current/Rent AVM/rent-avm/scraper/chromedriver.exe')

driver.get('https://www.aruodas.lt/butai-vilniuje-antakalnyje-pylimeliu-g-parduodamas-puikiai-irengtas-erdvus-ir-1-2860367/')
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')
driver.quit()

In [72]:
soup.find(class_='contacts-title')#.contents[0]

<div class="contacts-title">Pardavėjo kontaktai</div>

In [62]:
soup.find(class_='long-link-wrapper').contents[1]['href']

'http://www.rebaltic.lt'

In [91]:
with open('file.cfg', 'w') as f:
    f.writelines('\n'.join(['test1', 'test2', 'test3']))

In [92]:
with open('file.cfg', 'a') as f:
    f.writelines('\n'+'test1')

In [94]:
with open('file.cfg', 'r') as f:
    test = [line.strip() for line in f.readlines()]
    
print(test)

['test1', 'test2', 'test3', 'test1']


In [79]:
test

'test1\ntest2\ntest3'